In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from joblib import Parallel, delayed, parallel_backend
from tensorflow.keras.backend import clear_session

# Load the Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Major Project/creditcard.csv")
df.head(5)

# Scale Amount and Time

In [ ]:
df[['Scaled_Time', 'Scaled_Amount']] = StandardScaler().fit_transform(df[['Time', 'Amount']])
df = df.drop(['Time', 'Amount'], axis=1)

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

# Splitting the Dataset into Train and Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Creating the CNN Model

In [ ]:
def create_cnn(filters, kernel_size, learning_rate):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1], 1)),
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, activation='relu', padding='same'),
        layers.GlobalMaxPooling1D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.AdamW(learning_rate=learning_rate,weight_decay=1e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Function for Adaptive Mutation

In [ ]:
def adaptive_mutation(child, generation, max_generations, max_mutation=0.2):
    mutation_rate = max_mutation * (1 - (generation / max_generations))
    if random.random() < mutation_rate:
        child = (
            child[0] + random.randint(-8, 8),  # Mutate filters
            child[1] + random.randint(-1,-1),  # Keep kernel size
            child[2] + random.uniform(-0.001, 0.001)  # Change learning rate slightly
        )
    return child

# Function for Layer-Wise Crossover

In [ ]:
def cnn_layer_crossover(parent1, parent2):
    child = (
        parent2[0],  # Filters from parent2
        parent1[1],  # Kernel size from parent1
        (parent1[2] + parent2[2]) / 2  # Average learning rate
    )
    return child

# Function for Parallel Fitness Evaluation

In [ ]:
def parallel_fitness(population):
    def evaluate(individual):
        filters, kernel_size, learning_rate = individual
        model = create_cnn(int(filters), int(kernel_size), learning_rate)

        # Adding early stopping and learning rate scheduler
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

        model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0,
                  callbacks=[early_stopping, lr_scheduler], validation_split=0.1)

        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        return (accuracy, individual)

    results = Parallel(n_jobs=2)(delayed(evaluate)(ind) for ind in population)
    return sorted(results, key=lambda x: -x[0])

# Function for Initilization of Population

In [ ]:
def initialize_population(size):
    population = []
    for _ in range(size):
        filters = random.randint(16, 64)
        kernel_size = random.randint(2, 5)
        learning_rate = random.uniform(0.001, 0.01)
        population.append((filters, kernel_size, learning_rate))
    return population

# Genetic Algorithm Loop

In [ ]:
population_size = 20
generations = 10
population = initialize_population(population_size)

for generation in range(generations):
    print(f"Generation: {generation + 1}")
    fitness_results = parallel_fitness(population)

    # Select top individuals
    top_individuals = [ind for _, ind in fitness_results[:population_size // 2]]

    # Crossover to create new population
    new_population = []
    for i in range(0, len(top_individuals), 2):
        parent1 = top_individuals[i % len(top_individuals)]
        parent2 = top_individuals[(i + 1) % len(top_individuals)]
        child = cnn_layer_crossover(parent1, parent2)
        new_population.append(child)

    # Apply mutation
    new_population = [adaptive_mutation(child, generation, generations) for child in new_population]
    population = new_population

# Final Evaluation of Best Model

In [ ]:
best_model = create_cnn(*fitness_results[0][1])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)
y_predicted = (best_model.predict(X_test) > 0.5).astype(int)

# Accuracy, Confusion Matrix and Classification Report

In [ ]:
print('Classification report:\n', classification_report(y_test, y_predicted))
cm = confusion_matrix(y_true=y_test, y_pred=y_predicted)
print('Confusion matrix:\n', cm)
print('Accuracy:', accuracy_score(y_test, y_predicted))
precision = precision_score(y_test, y_predicted, pos_label=1.0)
print('precision : ',precision)
recall = recall_score(y_test, y_predicted, pos_label=1.0)
print('recall rate : ',recall)

# Saving the Model

In [ ]:
import os
output_path="/content/drive/MyDrive/Major Project/Model_Outputs"
os.makedirs(output_path,exist_ok=True)
#Saving the best model
model_save_path=os.path.join(output_path,"ns_best_model.keras")
best_model.save(model_save_path)
print("Model saved at:",model_save_path)
#Saving the predicted output
y_predicted=(best_model.predict(X_test)>0.5).astype(int)
#Saving the classification report and confusion matrix
report=classification_report(y_test,y_predicted)
cm=confusion_matrix(y_true=y_test,y_pred=y_predicted)
accuracy=accuracy_score(y_test,y_predicted)

output_file=os.path.join(output_path,"ns_model_metrics.txt")
with open(output_file, "w") as f:
  f.write("Classification Report:\n")
  f.write(report)
  f.write("\nConfusion Matrix:\n")
  f.write(str(cm))
  f.write(f"\nAccuracy:{accuracy:.4f}")
  f.write(f"\nPrecision:{precision:.4f}")
  f.write(f"\nRecall:{recall:.4f}")
print(f"Metrics saved at:{output_file}")
#saving confusion matrix as plot
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,6))
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
conf_matrix_path=os.path.join(output_path,'ns_confusion_matrix.png')
plt.savefig(conf_matrix_path)
print(f"Confusion matrix plot save at:{conf_matrix_path}")

# Loading the Saved Model

In [ ]:
import joblib
from joblib import load
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score, roc_auc_score,roc_curve,precision_recall_curve,auc,roc_curve,f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model

# Load the saved model from Google Drive
model_path = "/content/drive/MyDrive/Major Project/Model_Outputs/ns_best_model.keras"
loaded_model = load_model(model_path)

# Check the model architecture
loaded_model.summary()

# Use the loaded model to make predictions
y_pred = (loaded_model.predict(X_test) > 0.5).astype(int)

# Evaluate the model with test data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Classification Report:\n', classification_report(y_test, y_pred))
cm=confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
accuracy=accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1.0)
recall = recall_score(y_test, y_pred, pos_label=1.0)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
# AUC-ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.title('AUC-ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.grid()
plt.show()

# AUC-PRC Curve
precision_pr, recall_pr, _ = precision_recall_curve(y_test, y_pred)
pr_auc = auc(recall_pr, precision_pr)

plt.figure(figsize=(8, 6))
plt.plot(recall_pr, precision_pr, color='purple', lw=2, label=f'PR Curve (AUC = {pr_auc:.3f})')
plt.title('AUC-PRC Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower left")
plt.grid()
plt.show()